In [20]:
print("Importing...")
from pprint import pprint
import json
import sys
import os
from shutil import rmtree
from copy import deepcopy
from os.path import join
from importlib import reload

from os.path import isdir
from os.path import join
from os import mkdir
from shutil import rmtree

import numpy as np
import pandas as pd
from scipy import stats
from matplotlib import pyplot as plt
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import f1_score
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from keras import callbacks
from keras import losses
from keras import metrics
from keras import optimizers
from keras import layers
from keras import regularizers
import tensorflow_addons as tfa

sys.path.insert(0, os.path.join(os.getcwd(), "../scs"))
import scs_config as scsc
import data_loading as dl
import data_degrading as dd
import data_preparation as dp
import data_augmentation as da
import data_plotting as dplt
import learn
import lr_schedules
import hp_sets
import prepare_dataset

Importing...


In [2]:
R = 100
data_dir_original = "/home/2649/repos/SCS/data/"
dir_models = "/lustre/lrspec/users/2649/models/transformer_testing"

# Construct the directories if they don't exist or delete them and recreate
# them if they do and `restart_fit` is `True`.
dir_model = join(dir_models, f"{R}_dev")
dir_backup = join(dir_model, "backup")
dir_model_data = join(dir_model, "data")
if isdir(dir_model):
    rmtree(dir_model)
mkdir(dir_model)
mkdir(dir_backup)
mkdir(dir_model_data)

file_trn = join(dir_model_data, f"sn_data_trn.RPA.parquet")
file_tst = join(dir_model_data, f"sn_data_tst.RP.parquet")

In [3]:
hp = deepcopy(scsc.default_hyper_parameters)

hp["train_frac"] = 0.80
hp["noise_scale"] = 0.15848931924611134
hp["spike_scale"] = 1.045639552591273
hp["max_spikes"] = 3

In [ ]:
# Prepare the dataset from the original dataset dataframe `sn_data_file`.
sn_data_file = join(data_dir_original, "sn_data.parquet")
prepare_dataset.prepare_dataset(
    R,
    sn_data_file,
    dir_model_data,
    dir_model_data,
    dir_model_data,
    dir_model_data,
    hp["phase_range"],
    hp["ptp_range"],
    hp["wvl_range"],
    hp["train_frac"],
    hp["noise_scale"],
    hp["spike_scale"],
    hp["max_spikes"],
    random_state=hp["random_state"],
)

# Load the dataset.
df_trn = dl.load_sn_data(file_trn)
df_tst = dl.load_sn_data(file_tst)
dataset, num_wvl, num_classes = learn.prepare_datasets_for_training(
    df_trn, df_tst
)
Xtrn, Ytrn, Xtst, Ytst = dataset

Loaded: /home/2649/repos/SCS/data/sn_data.parquet
Artificially degrading the following dataset to a spectral resolution of 100:
    /home/2649/repos/SCS/data/sn_data.parquet


In [28]:
reload(learn)
model = learn.devmodel(
    num_wvls=Xtrn.shape[1],
    num_classes=num_classes,
    num_transformer_blocks=1,
    num_heads=4,
    key_dim=4,
    kr_l2=0,
    br_l2=0,
    ar_l2=0,
    dropout_attention=0,
    dropout_projection=0,
    filters=512,
    num_feed_forward_layers=3,
    feed_forward_layer_size=1024,
    dropout_feed_forward=0,
    initial_projection=100,
)

In [29]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 1, 139)]     0           []                               
                                                                                                  
 conv1d_15 (Conv1D)             (None, 100, 139)     200         ['input_7[0][0]']                
                                                                                                  
 multi_head_attention_6 (MultiH  (None, 100, 139)    9083        ['conv1d_15[0][0]',              
 eadAttention)                                                    'conv1d_15[0][0]']              
                                                                                                  
 dropout_24 (Dropout)           (None, 100, 139)     0           ['multi_head_attention_6[0]